In [1]:
import os
import sys
import pymysql
import re
import time
import pandas as pd
import numpy as np

from Utils.bulk_insert import BulkInsert

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
db_params = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': 'daigo1123',
    'database': 'dev_netkeiba',
    'port': 3306,
    'charset': 'utf8'
}
con = pymysql.connect(**db_params)

parameters = {
    'DATAFRAME_COL_NAMES': {
        'RACE_MASTER_TABLE_RESULT_INFO_COLS': [
            'race_id',
            'race_title',
            'race_course',
            'race_weather',
            'race_condition',
            'race_year',
            'race_month',
            'race_date',
            'race_dow',
            'starting_time',
            'race_info_1',
            'bracket_num',
            'horse_num',
            'href_to_horse',
            'horse_age',
            'horse_sex',
            'weight_penalty',
            'href_to_jockey',
            'href_to_owner',
            'horse_weight',
            'horse_weight_increment',
            'win_odds',
            'popularity_order',
            'arrival_sec_diff_from_first',
            'arrival_order'
        ],
       'RACE_PAST_X_RESULT_INFO_COLS': [
            'race_id',
            'bracket_num',
            'horse_num',
            'avg_past_x_arrival_order',
            'avg_arrival_sec_diff_from_first'
        ]
    }
}

queries = {
    'RACE_MASTER_TABLE_RESULT_INFO' :
        '''
        SELECT 
        race_id, race_title, race_course, race_weather, race_condition, race_year, race_month, race_date, race_dow, starting_time, race_info_1
        , bracket_num, horse_num, href_to_horse, horse_age, horse_sex
        , CAST(weight_penalty AS FLOAT) AS weight_penalty
        , href_to_jockey, href_to_owner
        , CAST(horse_weight AS FLOAT) AS horse_weight
        , CAST(horse_weight_increment AS FLOAT) AS horse_weight_increment
        , CAST(win_odds AS FLOAT)
        , popularity_order
        , CASE WHEN arrival_order <> 1 THEN CAST(arrival_sec_diff_from_first AS FLOAT) ELSE 0.000 END AS arrival_sec_diff_from_first
        , arrival_order
        FROM
        (
            SELECT 
            A.race_id, A.race_title, A.race_course, A.race_weather, A.race_condition, A.race_year, A.race_month, A.race_date, A.race_dow, A.starting_time, A.race_info_1
            , B.bracket_num, B.horse_num, B.href_to_horse, B.horse_age, B.horse_sex, B.weight_penalty, B.href_to_jockey, B.href_to_owner, B.horse_weight, B.horse_weight_increment
            , CAST(B.win_odds AS DECIMAL(5,1)) AS win_odds
            , CAST(B.popularity_order AS SIGNED) AS popularity_order
            , ROUND(TIMEDIFF(C.arrival_time, FIRST_VALUE(C.arrival_time) OVER (PARTITION BY C.race_id ORDER BY CAST(C.arrival_order AS SIGNED)))/60, 3)  AS arrival_sec_diff_from_first
            , CAST(C.arrival_order AS SIGNED) AS arrival_order
            FROM race_master AS A
            LEFT OUTER JOIN race_table_info AS B
            ON A.race_id = B.race_id
            LEFT OUTER JOIN race_result_info AS C
            ON A.race_id = C.race_id
            AND B.horse_num = C.horse_num
            WHERE 0=0
            AND C.arrival_order NOT IN ('')
            AND B.win_odds <> 0
        ) AS X
        ORDER BY race_id;
        ''',
    
    'RACE_PAST_X_RESULT_INFO':
        '''
        SELECT 
        race_id, bracket_num, horse_num
        , CAST(AVG(past_x_arrival_order) AS FLOAT) AS avg_past_x_arrival_order
        , CAST(AVG(arrival_sec_diff_from_first) AS FLOAT) AS avg_arrival_sec_diff_from_first
        FROM 
        (
            SELECT 
            A.race_id, A.bracket_num, A.horse_num, A.past_x, A.arrival_order AS past_x_arrival_order
            , B.arrival_time AS past_x_arrival_time
            , B.arrival_diff AS past_x_arrival_diff
            , B.arrival_sec_diff_from_first
            FROM race_past_5_result_info AS A
            LEFT OUTER JOIN (
            SELECT *
            , ROUND(TIMEDIFF(arrival_time, FIRST_VALUE(arrival_time) OVER (PARTITION BY race_id ORDER BY arrival_order))/60, 3) AS arrival_sec_diff_from_first
            FROM race_result_info
            ) AS B
            ON A.past_x_race_id = B.race_id
            AND A.arrival_order = B.arrival_order
            WHERE CAST(A.arrival_order AS SIGNED) NOT IN ('', 0)
        ) AS X
        WHERE past_x <= 3
        GROUP BY race_id, bracket_num, horse_num
        ORDER BY race_id, bracket_num, horse_num
        '''
}

## Extract Data from DB

In [4]:
def fetchall_and_make_list_by(query, con):
    try:
        cursor = con.cursor()
        cursor.execute(query)
        fetch_result = cursor.fetchall()
        fetch_result_list = [item for item in fetch_result]
        cursor.close()
        return fetch_result_list
    except Exception as e:
        print(e)

In [5]:
def get_race_master_table_result_data_frame(queries, parameters, con):
    selected_query = queries['RACE_MASTER_TABLE_RESULT_INFO']
    race_master_table_result_list = fetchall_and_make_list_by(selected_query, con)
    return pd.DataFrame(race_master_table_result_list, 
                                         columns=parameters['DATAFRAME_COL_NAMES']['RACE_MASTER_TABLE_RESULT_INFO_COLS'])

In [6]:
race_master_table_result_df = get_race_master_table_result_data_frame(queries, parameters, con)

In [7]:
race_master_table_result_df.shape

(570198, 25)

In [8]:
race_master_table_result_df.head()

,race_id,race_title,race_course,race_weather,race_condition,race_year,race_month,race_date,race_dow,starting_time,race_info_1,bracket_num,horse_num,href_to_horse,horse_age,horse_sex,weight_penalty,href_to_jockey,href_to_owner,horse_weight,horse_weight_increment,win_odds,popularity_order,arrival_sec_diff_from_first,arrival_order
0,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,4,6,https://db.netkeiba.com/horse/2006102194/,2,牝,54.0,https://db.netkeiba.com/jockey/00700/,https://db.netkeiba.com/trainer/01027/,468.0,0.0,4.3,2,0.000,1
1,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,8,14,https://db.netkeiba.com/horse/2006101653/,2,牡,54.0,https://db.netkeiba.com/jockey/00705/,https://db.netkeiba.com/trainer/01099/,460.0,0.0,5.2,4,0.007,2
2,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,1,1,https://db.netkeiba.com/horse/2006104966/,2,牡,54.0,https://db.netkeiba.com/jockey/00945/,https://db.netkeiba.com/trainer/01059/,442.0,2.0,3.1,1,0.008,3
3,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,7,11,https://db.netkeiba.com/horse/2006102534/,2,牡,54.0,https://db.netkeiba.com/jockey/00722/,https://db.netkeiba.com/trainer/00388/,482.0,-4.0,23.0,8,0.010,4
4,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,2,2,https://db.netkeiba.com/horse/2006110090/,2,牝,54.0,https://db.netkeiba.com/jockey/01084/,https://db.netkeiba.com/trainer/00419/,484.0,-20.0,21.9,6,0.010,5


In [9]:
def get_race_past_x_result_data_frame(queries, parameters, con):
    selected_query = queries['RACE_PAST_X_RESULT_INFO']
    race_race_past_x_result_list = fetchall_and_make_list_by(selected_query, con)
    return pd.DataFrame(race_race_past_x_result_list, 
                                         columns=parameters['DATAFRAME_COL_NAMES']['RACE_PAST_X_RESULT_INFO_COLS'])

In [10]:
race_past_x_result_df = get_race_past_x_result_data_frame(queries, parameters, con)

In [11]:
race_past_x_result_df.shape

(90900, 5)

In [12]:
race_past_x_result_df.head()

,race_id,bracket_num,horse_num,avg_past_x_arrival_order,avg_arrival_sec_diff_from_first
0,200801010101,1,1,3.0,NaN
1,200801010101,2,2,9.0,1.677
2,200801010101,3,3,2.5,0.991
3,200801010101,3,4,5.0,1.667
4,200801010101,4,5,9.0,15.002


In [13]:
training_race_df = pd.merge(race_master_table_result_df, race_past_x_result_df, 
                                    on=['race_id', 'bracket_num', 'horse_num'], how='left')

In [14]:
training_race_df.shape

(570198, 27)

In [15]:
training_race_df.head()

,race_id,race_title,race_course,race_weather,race_condition,race_year,race_month,race_date,race_dow,starting_time,race_info_1,bracket_num,horse_num,href_to_horse,horse_age,horse_sex,weight_penalty,href_to_jockey,href_to_owner,horse_weight,horse_weight_increment,win_odds,popularity_order,arrival_sec_diff_from_first,arrival_order,avg_past_x_arrival_order,avg_arrival_sec_diff_from_first
0,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,4,6,https://db.netkeiba.com/horse/2006102194/,2,牝,54.0,https://db.netkeiba.com/jockey/00700/,https://db.netkeiba.com/trainer/01027/,468.0,0.0,4.3,2,0.000,1,2.5,0.003
1,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,8,14,https://db.netkeiba.com/horse/2006101653/,2,牡,54.0,https://db.netkeiba.com/jockey/00705/,https://db.netkeiba.com/trainer/01099/,460.0,0.0,5.2,4,0.007,2,3.0,0.005
2,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,1,1,https://db.netkeiba.com/horse/2006104966/,2,牡,54.0,https://db.netkeiba.com/jockey/00945/,https://db.netkeiba.com/trainer/01059/,442.0,2.0,3.1,1,0.008,3,3.0,NaN
3,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,7,11,https://db.netkeiba.com/horse/2006102534/,2,牡,54.0,https://db.netkeiba.com/jockey/00722/,https://db.netkeiba.com/trainer/00388/,482.0,-4.0,23.0,8,0.010,4,6.0,1.675
4,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,2,2,https://db.netkeiba.com/horse/2006110090/,2,牝,54.0,https://db.netkeiba.com/jockey/01084/,https://db.netkeiba.com/trainer/00419/,484.0,-20.0,21.9,6,0.010,5,9.0,1.677


## EDA

In [16]:
from collections import Counter
from matplotlib import pyplot as plt
import pandas_profiling as pdp
from pprint import pprint

In [17]:
training_race_df.head()

,race_id,race_title,race_course,race_weather,race_condition,race_year,race_month,race_date,race_dow,starting_time,race_info_1,bracket_num,horse_num,href_to_horse,horse_age,horse_sex,weight_penalty,href_to_jockey,href_to_owner,horse_weight,horse_weight_increment,win_odds,popularity_order,arrival_sec_diff_from_first,arrival_order,avg_past_x_arrival_order,avg_arrival_sec_diff_from_first
0,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,4,6,https://db.netkeiba.com/horse/2006102194/,2,牝,54.0,https://db.netkeiba.com/jockey/00700/,https://db.netkeiba.com/trainer/01027/,468.0,0.0,4.3,2,0.000,1,2.5,0.003
1,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,8,14,https://db.netkeiba.com/horse/2006101653/,2,牡,54.0,https://db.netkeiba.com/jockey/00705/,https://db.netkeiba.com/trainer/01099/,460.0,0.0,5.2,4,0.007,2,3.0,0.005
2,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,1,1,https://db.netkeiba.com/horse/2006104966/,2,牡,54.0,https://db.netkeiba.com/jockey/00945/,https://db.netkeiba.com/trainer/01059/,442.0,2.0,3.1,1,0.008,3,3.0,NaN
3,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,7,11,https://db.netkeiba.com/horse/2006102534/,2,牡,54.0,https://db.netkeiba.com/jockey/00722/,https://db.netkeiba.com/trainer/00388/,482.0,-4.0,23.0,8,0.010,4,6.0,1.675
4,200801010101,２歳未勝利,芝1500m (右),曇,良,2008,8,16,土,10:40,1回札幌1日目 ２歳,2,2,https://db.netkeiba.com/horse/2006110090/,2,牝,54.0,https://db.netkeiba.com/jockey/01084/,https://db.netkeiba.com/trainer/00419/,484.0,-20.0,21.9,6,0.010,5,9.0,1.677


### TODO
- 欠損値の補完
    - avg_arrival_sec_diff_from_first: NULLとマイナスあり
    - arrival_sec_diff_from_first: NULLとマイナスあり
    - avg_past_x_arrival_order

In [18]:
def complement_missing_value(df):
    df['horse_weight'] = df['horse_weight'].replace(0, 470)
    return df

In [38]:
pprint(np.sort(pd.unique(training_race_df['avg_arrival_sec_diff_from_first'])))

array([-0.987   , -0.885667, -0.555667, ..., 35.003   , 69.2237  ,
             nan])


In [42]:
training_race_df['avg_arrival_sec_diff_from_first'].replace(np.nan, 100)

0           0.003000
1           0.005000
2         100.000000
3           1.675000
4           1.677000
5           0.991000
6           1.668000
7           2.164000
8          15.002000
9           2.657000
10          5.013000
11          1.667000
12          2.665000
13          3.000000
14          0.007000
15         18.337000
16          1.776000
17          2.333000
18          0.566333
19          0.990000
20          2.172500
21          1.676000
22          2.329330
23          3.338500
24          5.839500
25          0.887667
26          1.777670
27          0.669667
28          1.218330
29          1.447330
30          2.997500
31          2.000670
32          3.440330
33        100.000000
34          2.508500
35          6.675000
36          3.494500
37          3.216000
38        100.000000
39        100.000000
40        100.000000
41        100.000000
42        100.000000
43        100.000000
44        100.000000
45        100.000000
46        100.000000
47        100

In [48]:
training_race_df['avg_arrival_sec_diff_from_first'].where(training_race_df['avg_arrival_sec_diff_from_first'] >= 0, 50)

0          0.003000
1          0.005000
2         50.000000
3          1.675000
4          1.677000
5          0.991000
6          1.668000
7          2.164000
8         15.002000
9          2.657000
10         5.013000
11         1.667000
12         2.665000
13         3.000000
14         0.007000
15        18.337000
16         1.776000
17         2.333000
18         0.566333
19         0.990000
20         2.172500
21         1.676000
22         2.329330
23         3.338500
24         5.839500
25         0.887667
26         1.777670
27         0.669667
28         1.218330
29         1.447330
30         2.997500
31         2.000670
32         3.440330
33        50.000000
34         2.508500
35         6.675000
36         3.494500
37         3.216000
38        50.000000
39        50.000000
40        50.000000
41        50.000000
42        50.000000
43        50.000000
44        50.000000
45        50.000000
46        50.000000
47        50.000000
48        50.000000
49        50.000000


In [ ]:
def _make_target_variable(row, type_='odds_or_zero'):
    if type_ == 'odds_or_zero':
        if row['arrival_order'] == 1 or row['arrival_sec_diff_from_first'] <= 0.002:
            return row['win_odds']
        else:
            return 0

In [ ]:
def preprocess_target_variable(df):
    df['y']= df.apply(_make_target_variable, axis=1, type_='odds_or_zero')
    return df

In [ ]:
training_race_df = preprocess_target_variable(training_race_df)

#### Profiling to check finally

In [ ]:
# profile = pdp.ProfileReport(training_race_df)
# profile.to_file(output_file="Model/profile_report.html")
# profile